In [1]:
import gc
import torch
import torch.optim as optim
from torchinfo import summary

from KSI_models import KSI, ModifiedKSI, CAML
from KSI_utils import load_KSI_data, train_model, test_model

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
n_embedding = 100
n_hidden = 100 # 300 in paper, but too intensive for my machine
batch_size = 32
n_epochs = 25
save = True
profile = False
model_type = 'CAML'
early_stopping = 5

In [3]:
dir = 'data/original/'
loaders, wikivec, word_to_ix = load_KSI_data(dir=dir, 
                                             batch_size=batch_size, 
                                             train=True, 
                                             val=True, 
                                             test=True, 
                                             device=DEVICE)
train_dataloader = loaders['train']
val_dataloader = loaders['val']
test_dataloader = loaders['test']

n_wiki, n_vocab = wikivec.shape
n_words = len(word_to_ix)

In [4]:
# note_lengths = []
# for data in train_dataloader:
#     n, _, _ = data
#     note_lengths.append(n.shape[1])
# avg_note_size = np.round(np.array(note_lengths).mean()).astype(int)

avg_note_size = 2455

In [5]:
base_model = CAML(n_words, n_wiki, n_embedding, n_hidden)
base_model = base_model.to(DEVICE)
base_summary = summary(base_model, [(batch_size, avg_note_size), 
                                    (batch_size, n_vocab)], 
                       dtypes=[torch.int, torch.float])

base_summary

Layer (type:depth-idx)                   Output Shape              Param #
CAML                                     --                        --
├─Embedding: 1-1                         [32, 2455, 100]           4,796,200
├─Dropout: 1-2                           [32, 2455, 100]           --
├─Conv1d: 1-3                            [32, 100, 2456]           100,100
├─Linear: 1-4                            --                        34,400
├─Linear: 1-5                            --                        34,744
Total params: 4,965,444
Trainable params: 4,965,444
Non-trainable params: 0
Total mult-adds (G): 8.02
Input size (MB): 1.87
Forward/backward pass size (MB): 125.72
Params size (MB): 19.86
Estimated Total Size (MB): 147.46

In [6]:
optimizer = optim.Adam(base_model.parameters())
prof_base = train_model(base_model, 
                        train_dataloader=train_dataloader,
                        val_dataloader=val_dataloader,
                        optimizer=optimizer,
                        n_epochs=n_epochs,
                        profile=profile, 
                        log_path=f'./log/{model_type}',
                        device=DEVICE,
                        early_stopping=early_stopping)

Epoch: 001, Train Recall@10: 0.6281, Val Recall@10: 0.6273, Train Micro F1: 0.5048, Val Micro F1: 0.4980, Train Macro F1: 0.0409, Val Macro F1: 0.0486, Train Micro AUC: 0.9539, Val Micro AUC: 0.9428, Train Macro AUC: 0.6861, Val Macro AUC: 0.6866
Epoch: 002, Train Recall@10: 0.7344, Val Recall@10: 0.7252, Train Micro F1: 0.6097, Val Micro F1: 0.5883, Train Macro F1: 0.0837, Val Macro F1: 0.0957, Train Micro AUC: 0.9674, Val Micro AUC: 0.9583, Train Macro AUC: 0.7714, Val Macro AUC: 0.7556
Epoch: 003, Train Recall@10: 0.7776, Val Recall@10: 0.7622, Train Micro F1: 0.6525, Val Micro F1: 0.6225, Train Macro F1: 0.1186, Val Macro F1: 0.1333, Train Micro AUC: 0.9740, Val Micro AUC: 0.9649, Train Macro AUC: 0.8187, Val Macro AUC: 0.7834
Epoch: 004, Train Recall@10: 0.8022, Val Recall@10: 0.7779, Train Micro F1: 0.6758, Val Micro F1: 0.6347, Train Macro F1: 0.1498, Val Macro F1: 0.1586, Train Micro AUC: 0.9782, Val Micro AUC: 0.9687, Train Macro AUC: 0.8525, Val Macro AUC: 0.8008
Epoch: 005, 

In [7]:
if save:
    torch.save(base_model, f'{dir}{model_type}_model.pt')
if profile:
    print(prof_base.key_averages(group_by_stack_n=5).table(sort_by='self_cuda_time_total'))

In [8]:
tt_recall_at_k, tt_micro_f1, tt_macro_f1, tt_micro_auc, tt_macro_auc, label_aucs_base = test_model(base_model, 
                                                                                                   test_dataloader, 
                                                                                                   wikivec,
                                                                                                   device=DEVICE)
print(f'Test Recall@10: {tt_recall_at_k:.4f}, Test Micro F1: {tt_micro_f1:.4f}, Test Macro F1: {tt_macro_f1:.4f}' +
      f', Test Micro AUC: {tt_micro_auc:.4f}, Test Macro AUC: {tt_macro_auc:.4f}')
del base_model
gc.collect()
if DEVICE == 'cuda':
    torch.cuda.empty_cache()

Test Recall@10: 0.8043, Test Micro F1: 0.6585, Test Macro F1: 0.2427, Test Micro AUC: 0.9763, Test Macro AUC: 0.8351


In [9]:
ksi = KSI(n_embedding, n_vocab)
ksi.to(DEVICE)
model = CAML(n_words, n_wiki, n_embedding, n_hidden, ksi=ksi)
model = model.to(DEVICE)
ksi_summary = summary(model, [(batch_size, avg_note_size), 
                              (batch_size, n_vocab),
                              (n_wiki, n_vocab)], 
                      dtypes=[torch.int, torch.float, torch.float])

ksi_summary

Layer (type:depth-idx)                   Output Shape              Param #
CAML                                     --                        --
├─KSI: 1-1                               --                        --
│    └─Linear: 2-1                       --                        (recursive)
│    └─Linear: 2-2                       --                        (recursive)
│    └─Linear: 2-3                       --                        (recursive)
├─Embedding: 1-2                         [32, 2455, 100]           4,796,200
├─Dropout: 1-3                           [32, 2455, 100]           --
├─Conv1d: 1-4                            [32, 100, 2456]           100,100
├─KSI: 1-1                               --                        --
│    └─Linear: 2-4                       [32, 344, 100]            1,217,400
│    └─Linear: 2-5                       [32, 344, 100]            10,100
│    └─Linear: 2-6                       [32, 344, 1]              101
Total params: 6,123,901
Trainable 

In [10]:
optimizer = optim.Adam(model.parameters())
prof_ksi = train_model(model, 
                       train_dataloader=train_dataloader,
                       val_dataloader=val_dataloader,
                       wikivec=wikivec,
                       optimizer=optimizer,
                       n_epochs=n_epochs, 
                       profile=profile, 
                       log_path=f'./log/{model_type}_KSI',
                       device=DEVICE,
                       early_stopping=early_stopping)

Epoch: 001, Train Recall@10: 0.7117, Val Recall@10: 0.7044, Train Micro F1: 0.5164, Val Micro F1: 0.5058, Train Macro F1: 0.0766, Val Macro F1: 0.0893, Train Micro AUC: 0.9722, Val Micro AUC: 0.9645, Train Macro AUC: 0.8515, Val Macro AUC: 0.8304
Epoch: 002, Train Recall@10: 0.7777, Val Recall@10: 0.7671, Train Micro F1: 0.6137, Val Micro F1: 0.5923, Train Macro F1: 0.1256, Val Macro F1: 0.1382, Train Micro AUC: 0.9795, Val Micro AUC: 0.9716, Train Macro AUC: 0.9016, Val Macro AUC: 0.8568
Epoch: 003, Train Recall@10: 0.8096, Val Recall@10: 0.7910, Train Micro F1: 0.6594, Val Micro F1: 0.6288, Train Macro F1: 0.1734, Val Macro F1: 0.1830, Train Micro AUC: 0.9832, Val Micro AUC: 0.9747, Train Macro AUC: 0.9269, Val Macro AUC: 0.8734
Epoch: 004, Train Recall@10: 0.8275, Val Recall@10: 0.7998, Train Micro F1: 0.6818, Val Micro F1: 0.6401, Train Macro F1: 0.2162, Val Macro F1: 0.2147, Train Micro AUC: 0.9856, Val Micro AUC: 0.9762, Train Macro AUC: 0.9428, Val Macro AUC: 0.8807
Epoch: 005, 

In [11]:
if save:
    torch.save(model, f'{dir}{model_type}_KSI_model.pt')
if profile:
    print(prof_ksi.key_averages(group_by_stack_n=5).table(sort_by='self_cuda_time_total'))

In [12]:
tt_recall_at_k, tt_micro_f1, tt_macro_f1, tt_micro_auc, tt_macro_auc, label_aucs_ksi = test_model(model, 
                                                                                                  test_dataloader, 
                                                                                                  wikivec,
                                                                                                  device=DEVICE)
print(f'Test Recall@10: {tt_recall_at_k:.4f}, Test Micro F1: {tt_micro_f1:.4f}, Test Macro F1: {tt_macro_f1:.4f}' +
      f', Test Micro AUC: {tt_micro_auc:.4f}, Test Macro AUC: {tt_macro_auc:.4f}')
del model
gc.collect()
if DEVICE == 'cuda':
    torch.cuda.empty_cache()

Test Recall@10: 0.8029, Test Micro F1: 0.6452, Test Macro F1: 0.2364, Test Micro AUC: 0.9785, Test Macro AUC: 0.8912


In [13]:
# run modified KSI using frequency vectors rather than binary vectors
dir = 'data/original_freqs/'
loaders, wikivec, word_to_ix = load_KSI_data(dir=dir, 
                                             batch_size=batch_size, 
                                             train=True, 
                                             val=True, 
                                             test=True, 
                                             device=DEVICE)
train_dataloader = loaders['train']
val_dataloader = loaders['val']
test_dataloader = loaders['test']

n_wiki, n_vocab = wikivec.shape
n_words = len(word_to_ix)

In [14]:
mod_ksi = ModifiedKSI(n_embedding, n_vocab)
mod_ksi.to(DEVICE)
mod_model = CAML(n_words, n_wiki, n_embedding, n_hidden, ksi=mod_ksi)
mod_model = mod_model.to(DEVICE)
mod_summary = summary(mod_model, [(batch_size, avg_note_size), 
                                  (batch_size, n_vocab),
                                  (n_wiki, n_vocab)], 
                      dtypes=[torch.int, torch.float, torch.float])

mod_summary

Layer (type:depth-idx)                   Output Shape              Param #
CAML                                     --                        --
├─ModifiedKSI: 1-1                       --                        --
│    └─Linear: 2-1                       --                        (recursive)
│    └─Linear: 2-2                       --                        (recursive)
│    └─Linear: 2-3                       --                        (recursive)
│    └─Linear: 2-4                       --                        (recursive)
├─Embedding: 1-2                         [32, 2455, 100]           4,796,200
├─Dropout: 1-3                           [32, 2455, 100]           --
├─Conv1d: 1-4                            [32, 100, 2456]           100,100
├─ModifiedKSI: 1-1                       --                        --
│    └─Linear: 2-5                       [32, 344, 12173, 1]       2
│    └─Linear: 2-6                       [32, 344, 100]            1,217,400
│    └─Linear: 2-7             

In [15]:
optimizer = optim.Adam(mod_model.parameters())
prof_mod_ksi = train_model(mod_model, 
                           train_dataloader=train_dataloader,
                           val_dataloader=val_dataloader,
                           wikivec=wikivec,
                           optimizer=optimizer,
                           n_epochs=n_epochs, 
                           profile=profile, 
                           log_path=f'./log/{model_type}_ModifiedKSI',
                           device=DEVICE,
                           early_stopping=early_stopping)

Epoch: 001, Train Recall@10: 0.7414, Val Recall@10: 0.7404, Train Micro F1: 0.5554, Val Micro F1: 0.5498, Train Macro F1: 0.0940, Val Macro F1: 0.1128, Train Micro AUC: 0.9757, Val Micro AUC: 0.9699, Train Macro AUC: 0.8647, Val Macro AUC: 0.8641
Epoch: 002, Train Recall@10: 0.7822, Val Recall@10: 0.7731, Train Micro F1: 0.6041, Val Micro F1: 0.5916, Train Macro F1: 0.1386, Val Macro F1: 0.1616, Train Micro AUC: 0.9803, Val Micro AUC: 0.9745, Train Macro AUC: 0.8950, Val Macro AUC: 0.8782
Epoch: 003, Train Recall@10: 0.8058, Val Recall@10: 0.7912, Train Micro F1: 0.6370, Val Micro F1: 0.6137, Train Macro F1: 0.1883, Val Macro F1: 0.1982, Train Micro AUC: 0.9829, Val Micro AUC: 0.9766, Train Macro AUC: 0.9107, Val Macro AUC: 0.8868
Epoch: 004, Train Recall@10: 0.8195, Val Recall@10: 0.7985, Train Micro F1: 0.6593, Val Micro F1: 0.6278, Train Macro F1: 0.2176, Val Macro F1: 0.2280, Train Micro AUC: 0.9847, Val Micro AUC: 0.9778, Train Macro AUC: 0.9291, Val Macro AUC: 0.8972
Epoch: 005, 

In [16]:
if save:
    torch.save(mod_model, f'{dir}{model_type}_ModifiedKSI_model.pt')
if profile:
    print(prof_mod_ksi.key_averages(group_by_stack_n=5).table(sort_by='self_cuda_time_total'))

In [17]:
tt_recall_at_k, tt_micro_f1, tt_macro_f1, tt_micro_auc, tt_macro_auc, label_aucs_mod = test_model(mod_model, 
                                                                                                  test_dataloader, 
                                                                                                  wikivec,
                                                                                                  device=DEVICE)
print(f'Test Recall@10: {tt_recall_at_k:.4f}, Test Micro F1: {tt_micro_f1:.4f}, Test Macro F1: {tt_macro_f1:.4f}' +
      f', Test Micro AUC: {tt_micro_auc:.4f}, Test Macro AUC: {tt_macro_auc:.4f}')
del mod_model
gc.collect()
if DEVICE == 'cuda':
    torch.cuda.empty_cache()

Test Recall@10: 0.8067, Test Micro F1: 0.6480, Test Macro F1: 0.2777, Test Micro AUC: 0.9799, Test Macro AUC: 0.9011


In [18]:
# run modified KSI using tfidf vectors rather than binary vectors
dir = 'data/original_tfidf/'
loaders, wikivec, word_to_ix = load_KSI_data(dir=dir, 
                                             batch_size=batch_size, 
                                             train=True, 
                                             val=True, 
                                             test=True, 
                                             device=DEVICE)
train_dataloader = loaders['train']
val_dataloader = loaders['val']
test_dataloader = loaders['test']

n_wiki, n_vocab = wikivec.shape
n_words = len(word_to_ix)

In [19]:
mod_ksi2 = ModifiedKSI(n_embedding, n_vocab)
mod_ksi2.to(DEVICE)
tfidf_model = CAML(n_words, n_wiki, n_embedding, n_hidden, ksi=mod_ksi2)
tfidf_model = tfidf_model.to(DEVICE)
tfidf_summary = summary(tfidf_model, [(batch_size, avg_note_size), 
                                      (batch_size, n_vocab),
                                      (n_wiki, n_vocab)], 
                        dtypes=[torch.int, torch.float, torch.float])

tfidf_summary

Layer (type:depth-idx)                   Output Shape              Param #
CAML                                     --                        --
├─ModifiedKSI: 1-1                       --                        --
│    └─Linear: 2-1                       --                        (recursive)
│    └─Linear: 2-2                       --                        (recursive)
│    └─Linear: 2-3                       --                        (recursive)
│    └─Linear: 2-4                       --                        (recursive)
├─Embedding: 1-2                         [32, 2455, 100]           4,796,200
├─Dropout: 1-3                           [32, 2455, 100]           --
├─Conv1d: 1-4                            [32, 100, 2456]           100,100
├─ModifiedKSI: 1-1                       --                        --
│    └─Linear: 2-5                       [32, 344, 12173, 1]       2
│    └─Linear: 2-6                       [32, 344, 100]            1,217,400
│    └─Linear: 2-7             

In [20]:
optimizer = optim.Adam(tfidf_model.parameters())
prof_tfidf_ksi = train_model(tfidf_model, 
                             train_dataloader=train_dataloader,
                             val_dataloader=val_dataloader,
                             wikivec=wikivec,
                             optimizer=optimizer,
                             n_epochs=n_epochs, 
                             profile=profile, 
                             log_path=f'./log/{model_type}_ModifiedKSI_tfidf',
                             device=DEVICE,
                             early_stopping=early_stopping)

Epoch: 001, Train Recall@10: 0.7436, Val Recall@10: 0.7413, Train Micro F1: 0.5555, Val Micro F1: 0.5510, Train Macro F1: 0.1081, Val Macro F1: 0.1323, Train Micro AUC: 0.9763, Val Micro AUC: 0.9705, Train Macro AUC: 0.8720, Val Macro AUC: 0.8654
Epoch: 002, Train Recall@10: 0.7893, Val Recall@10: 0.7790, Train Micro F1: 0.6127, Val Micro F1: 0.6002, Train Macro F1: 0.1687, Val Macro F1: 0.1938, Train Micro AUC: 0.9813, Val Micro AUC: 0.9754, Train Macro AUC: 0.8979, Val Macro AUC: 0.8894
Epoch: 003, Train Recall@10: 0.8102, Val Recall@10: 0.7955, Train Micro F1: 0.6424, Val Micro F1: 0.6203, Train Macro F1: 0.2121, Val Macro F1: 0.2183, Train Micro AUC: 0.9836, Val Micro AUC: 0.9773, Train Macro AUC: 0.9173, Val Macro AUC: 0.8935
Epoch: 004, Train Recall@10: 0.8234, Val Recall@10: 0.8039, Train Micro F1: 0.6632, Val Micro F1: 0.6287, Train Macro F1: 0.2427, Val Macro F1: 0.2382, Train Micro AUC: 0.9853, Val Micro AUC: 0.9783, Train Macro AUC: 0.9325, Val Macro AUC: 0.8991
Epoch: 005, 

In [21]:
if save:
    torch.save(tfidf_model, f'{dir}{model_type}_ModifiedKSI_tfidf_model.pt')
if profile:
    print(prof_tfidf_ksi.key_averages(group_by_stack_n=5).table(sort_by='self_cuda_time_total'))

In [22]:
tt_recall_at_k, tt_micro_f1, tt_macro_f1, tt_micro_auc, tt_macro_auc, label_aucs_mod = test_model(tfidf_model, 
                                                                                                  test_dataloader, 
                                                                                                  wikivec,
                                                                                                  device=DEVICE)
print(f'Test Recall@10: {tt_recall_at_k:.4f}, Test Micro F1: {tt_micro_f1:.4f}, Test Macro F1: {tt_macro_f1:.4f}' +
      f', Test Micro AUC: {tt_micro_auc:.4f}, Test Macro AUC: {tt_macro_auc:.4f}')
del tfidf_model
gc.collect()
if DEVICE == 'cuda':
    torch.cuda.empty_cache()

Test Recall@10: 0.8076, Test Micro F1: 0.6414, Test Macro F1: 0.2678, Test Micro AUC: 0.9805, Test Macro AUC: 0.9037
